In [1]:
print("Let's Go!" )
from PIL import Image
import torch
from torchvision import transforms
import os
import shutil
import numpy as np
from torch.optim import Adam
import random 
from torch.nn import ReLU
from torch.nn import LayerNorm
import torch.nn.functional as F
from torch.nn.init import xavier_uniform_
from torch.nn import BatchNorm2d
from torch.nn import Parameter
from torch.nn import Linear
from torch.nn import DataParallel
from torch.nn import Sequential
from torch.nn import ConvTranspose2d
from torch.nn import GRUCell
from torch.nn import Module
from torch.nn import Flatten
from torch.nn.init import normal_
from torch.utils.data import Dataset, DataLoader
from torch.nn.functional import softmax
from torch.nn import Conv2d
from torch.nn import Conv1d
from torch.nn import MSELoss
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import CosineAnnealingLR
import h5py
import sys
from tqdm import tqdm
from collections import OrderedDict
from torchvision.transforms import InterpolationMode
import matplotlib.pyplot as plt



torch.manual_seed(0)
np.random.seed(0)

print("torch version:" , torch.__version__)

Let's Go!
torch version: 2.1.2


In [2]:
images_val = '/kaggle/input/clevertexv11-val-images/CLEVERTex10_val_images'
clever6_indices_path = '/kaggle/input/clevertex6-val/CLEVERTex6_val/CLEVRTex6_val_indices.npy'
val_cluster_path = '/kaggle/input/clevertex6-val/CLEVERTex6_val/val_masks_clustering.npy'
clever6_indices = np.load(clever6_indices_path)
clever11_indices = np.array([i + 40000 for i in range(10000)])

class ImageDatasetSlotAttention(Dataset):
    def __init__(self, images_folder , indices , transform , cluster_path):
        super(ImageDatasetSlotAttention , self).__init__()
        self.images_folder = images_folder
        self.folder = images_folder
        self.transform = transform
        self.indices = indices
        self.all_clusters = np.load(cluster_path)
        print("indices shape:" , self.indices.shape)
        print("clusters data shape:" , self.all_clusters.shape)
        self.all_data =  self.build_data(images_folder)
        print("data size:" , len(self.all_data))

        del self.all_clusters
        del self.transform 

    def build_data(self , folder_path):
        files = sorted(os.listdir(folder_path))
        files = sorted(files)
        data = []
        print("Loading data!")
        for idx in tqdm(self.indices):
            
            image_path = self.images_folder + f'/CLEVRTEX_full_{idx:06d}.png'
            image = Image.open(image_path).convert('RGB')
            cluster = torch.tensor(self.all_clusters[idx - 40000])
            data.append((self.transform(image) , cluster))
            
        print("Loaded data!")
        return data


    def __len__(self):
        return len(self.indices)
    def __getitem__(self , idx):
        return self.all_data[idx]

transform = transforms.Compose([
  transforms.PILToTensor(),
  transforms.ConvertImageDtype(torch.float),
  transforms.Normalize((0.5 ,  0.5 , 0.5), (0.5, 0.5 , 0.5)),
])


val_data = ImageDatasetSlotAttention(images_folder = images_val , indices = clever11_indices,
                                    transform = transform, cluster_path = val_cluster_path)
print("image tensor max-min" , val_data[0][0].max().item() , val_data[0][0].min().item())
print("cluster tensor max-min" , val_data[4][1].max().item() , val_data[4][1].min().item())


indices shape: (10000,)
clusters data shape: (10000, 128, 128)
Loading data!


100%|██████████| 10000/10000 [00:52<00:00, 191.13it/s]

Loaded data!
data size: 10000
image tensor max-min 0.9686274528503418 -1.0
cluster tensor max-min 9 0


In [3]:
class Block(Module):
    def __init__(self, channels):
        super(Block, self).__init__()
        self.downsample = Sequential(OrderedDict([
            ('conv1' , Conv2d(channels, channels, kernel_size = 5 , stride = 1 , padding = 2)),
            ('bn1'   , BatchNorm2d(channels)),
            ('relu1' , ReLU()),
            ('conv2' , Conv2d(channels, channels, kernel_size = 5 , stride = 1 , padding = 2)),
            ('bn2'   , BatchNorm2d(channels))
        ]))
        
        self.relu = ReLU()

    def forward(self, x):
        return self.relu(x + self.downsample(x))

class BlockUp(Module):
    def __init__(self , in_channels , out_channels):
        super(BlockUp, self).__init__()
        
        self.downsample = Sequential(OrderedDict([
            ('conv1' , Conv2d(in_channels , out_channels , kernel_size = 5 , stride = 1 , padding = 2)),
            ('bn1'   , BatchNorm2d(out_channels)),
            ('relu1' , ReLU()),
            ('conv2' , Conv2d(out_channels , out_channels , kernel_size = 5 , stride = 1 , padding = 2)),
            ('bn2'   , BatchNorm2d(out_channels)),
        ]))
        self.skip = Sequential(OrderedDict([
            ('conv1' , Conv2d(in_channels , out_channels, kernel_size = 5 , stride = 1 , padding = 2)),
            ('bn1'   , BatchNorm2d(out_channels))
        ]))
        self.relu = ReLU()



    def forward(self, x):
        return self.relu(self.skip(x) + self.downsample(x))
    
    
class ResNet18(Module):
    def __init__(self):

        super(ResNet18, self).__init__()
        layer0 = Sequential(OrderedDict([
         ('conv' , Conv2d(3 , 16 , kernel_size = 5 , stride = 1 , padding = 2)),
         ('bn'   , BatchNorm2d(16)),
         ('relu' , ReLU())
        ]))

        self.resnet = Sequential(OrderedDict([
            ('layer0' , layer0),
            ('block1' , Block(16)),
            ('block2' , Block(16)),
            ('block3' , BlockUp(16 , 32)),
            ('block4' , Block(32)),
            ('block5' , Block(32)),
            ('block6' , BlockUp(32 , 64)),
            ('block7' , Block(64)),
            ('block8' , Block(64))
        ]))


    def forward(self, x):
        return self.resnet(x)


In [4]:


class ObjectDiscovery(Module):
    def __init__(self , encoder_resolution ,  decoder_resolution , T   , K , D_slots):
        super(ObjectDiscovery , self).__init__()
        print(f'Initialized ObjectDiscovery!')
        self.layers = Sequential(OrderedDict([
          ("encoder" , ImageEncoder(resolution = encoder_resolution , T = T,
                                    K = K , D_slots = D_slots )),
          ("decoder" , SlotAttentionDecoder(resolution = decoder_resolution , K = K , D_slots = D_slots))
        ]))
    def forward(self , image):
        return self.layers(image)

class PositionEncoder(Module):
    def __init__(self, output_dim , resolution):
        super(PositionEncoder , self).__init__()
        self.linear =  Linear(in_features = 4 , out_features = output_dim)
        # above is equivalent to a linear layer
        self.grid = Parameter(data = PositionEncoder.build_grid(resolution) , requires_grad = False)
        print("Grid shape:" , self.grid.shape)

    @staticmethod
    def build_grid(resolution):
        ranges = [np.linspace(start = 0.0 , stop = 1.0 , num = dimension) for dimension in resolution] # dim = (2 , 128)
        grid = np.meshgrid(*ranges , sparse = False, indexing = "ij") # dim = (128 , 128)
        # row[i] of grid[0] has all elements i / 127 and col[j] of grid[1] has all elements j / 127
        grid = np.stack(grid , axis = -1) # dim = (64 , 64 , 2) to match conv later
        grid = np.expand_dims(grid, axis = 0) # dim = (1 , 64 , 64 , 2) for batch dimension
        grid = grid.astype(np.float32) # PyTorch throws an error later otherwise
        return torch.tensor(np.concatenate([grid , 1.0 - grid] , axis = 3)) # (1 , 64 , 64 , 4)


    def forward(self, x): # x has shape (batch , 64 , 64 , D_inputs)
        return x + self.linear(self.grid)


class ImageEncoder(Module):
    def __init__(self , resolution , T  ,  K , D_slots):
        super(ImageEncoder , self).__init__()
        print(f'Initialized ImageEncoder! resolution: {resolution}')
        D_inputs = 64
        self.encoder_cnn = ResNet18()
        down_resolution = (128 , 128)
        positional_encoder = PositionEncoder(output_dim =  D_inputs , resolution = down_resolution)


        slot_attention = SlotAttention(T = T , K = K , D_slots = D_slots)
        self.pos_encode_feedforward_slotattn = Sequential(OrderedDict([
            ("pos_encoder" , positional_encoder), #  (batch , 64 , 64 , D_inputs)
            ("flatten" , Flatten(start_dim = 1 , end_dim = 2)), # (batch , 64 * 64 , D_inputs)
            ("layer_norm" ,  LayerNorm(normalized_shape = 64)), # (batch , 64 * 64 , D_inputs)
            ("linear1:" , Linear(in_features = 64 , out_features = 128)),
            ("relu1:"   , ReLU()),
            ("linear2:" , Linear(in_features = 128 , out_features = 128)),
            ("slot_attention" , slot_attention) #  (batch , K , D_slots)
        ]))
    # feature map has shape (channels , h , w)
    # N is h * w i.e. each pixel is a different feature "vector", size of this vector of D_inputs = # of channels
    # channels = D_inputs , h * w = N
    def forward(self , x): # x is aimge with shape (batch , 3 , 126 , 128)
        x = self.encoder_cnn(x).permute(dims = (0 , 2 , 3 , 1))  # (batch , 64 , 64 , D_inputs)
        return self.pos_encode_feedforward_slotattn(x)



class SlotAttention(Module):
    def __init__(self , T, K , D_slots  , epsilon = 1e-8):
        super(SlotAttention , self).__init__()
        print(f'Initialzing SlotAttention parameters: T: {T} , K: {K} , D_slots: {D_slots}')
        self.T = T
        self.K = K
        self.D_slots = D_slots
        self.epsilon = epsilon

        self.norm_inputs = LayerNorm(normalized_shape =  128)
        self.query_from_slots  = Sequential(OrderedDict([
            ("NormSlots" , LayerNorm(normalized_shape = D_slots)),
            ("SlotQuery" , Linear(in_features = D_slots  , out_features = D_slots , bias = False))
        ]))


#         self.slots_init = Parameter(data = normal_(torch.empty(1 , K , D_slots)) , requires_grad = True)

        self.init_latents = Parameter(normal_(torch.empty(1 , self.K , self.D_slots)))



        self.keys_from_inputs  = Linear(in_features = 128 , out_features = D_slots , bias = False)
        self.vals_from_inputs  = Linear(in_features = 128 , out_features = D_slots , bias = False)
        self.gru = GRUCell(input_size = D_slots , hidden_size = D_slots)
        self.norm_feed = Sequential(OrderedDict([
            ("layer_norm" ,  LayerNorm(normalized_shape = D_slots)),
            ("linear1" , Linear(in_features = D_slots , out_features = 256)),
            ("relu1"   , ReLU()),
            ("linear2" , Linear(in_features = 256 , out_features = D_slots))
        ]))


    def forward(self, inputs):
        batch_size = inputs.shape[0]
        inputs = self.norm_inputs(inputs)
        keys = self.keys_from_inputs(inputs)
        vals = self.vals_from_inputs(inputs)
        slots = self.init_latents.repeat(batch_size , 1 , 1)
        for _ in range(self.T):
            slots_prev = slots
            queries = self.query_from_slots(slots)
            attn_logits = (self.D_slots ** -0.5) * torch.einsum('bnc,bmc->bnm', keys , queries)
            attn = F.softmax(attn_logits , dim = -1)
            attn = attn + self.epsilon
            attn = attn / torch.sum(attn, dim = 1, keepdim = True)
            updates = torch.einsum('bnm,bnc->bmc' , attn , vals)
            slots = self.gru(
                updates.view(batch_size * self.K , self.D_slots),
                slots_prev.view(batch_size * self.K , self.D_slots),
            )
            slots = slots.view(batch_size , self.K , self.D_slots)
            slots = slots + self.norm_feed(slots)

        return slots


class SlotAttentionDecoder(Module):
    def __init__(self , resolution , K , D_slots):
        super(SlotAttentionDecoder , self).__init__()
        print(f'Initialized SlotAttentionDecoder! resolution: {resolution}')
        print(f'Decoder parameters: K: {K} , D_slots: {D_slots}')
        self.resolution = resolution
        self.positional_encoder = PositionEncoder(output_dim = D_slots , resolution = resolution)

        self.K = K
        self.D_slots = D_slots

        self.decoder_cnn = Sequential(OrderedDict([
            ("conv1" , ConvTranspose2d(in_channels = D_slots , out_channels = 64 , kernel_size = 5 , stride = 2 , padding = 2 , output_padding = 1)),
            ('relu1' , ReLU()),
            ("conv2" , ConvTranspose2d(in_channels = 64 , out_channels = 64 , kernel_size = 5 , stride = 2 , padding = 2 , output_padding = 1)),
            ('relu2' , ReLU()),
            ("conv3" , ConvTranspose2d(in_channels = 64 , out_channels = 64 , kernel_size = 5 , stride = 2 , padding = 2 , output_padding = 1)),
            ('relu3' , ReLU()),
            ("conv4" , ConvTranspose2d(in_channels = 64 , out_channels = 64 , kernel_size = 5 , stride = 2 , padding = 2 , output_padding = 1)),
            ('relu4' , ReLU()),
            ("conv5" , ConvTranspose2d(in_channels = 64 , out_channels = 4 , kernel_size = 1 , stride = 1)),
        ]))


    def forward(self , slots):
        # slots comes form the ImageEncoder shape : (batch , K , D_slots)
        batch_size = slots.shape[0]
        x =  slots.view(batch_size * self.K , 1 , 1 , self.D_slots)
        x = x.repeat(1 , self.resolution[0] , self.resolution[1] , 1)
        x = self.positional_encoder(x).permute(0 , 3 , 1 , 2)
        x = self.decoder_cnn(x) # dim = (batch * K , 4 , 128 , 128)
        x = x.view(batch_size, self.K , 4 , 128 , 128)# dim = (batch , K , 4 , 128 , 128)
        recons , masks = x[: , : , : 3 , : , :]  , x[: , : , -1 : , : , :]
        masks = F.softmax(masks , dim = 1) # dim = (batch , K , 1 , 128, 128)
        reconstructed = torch.sum(recons * masks , dim = 1) # dim = (batch , 3 , 128, 128)
        return reconstructed , recons , masks

In [5]:
model_path = '/kaggle/input/v17-model-180/model_180'
print("Loading model!")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:" , device)
model_dict = torch.load(model_path)
model = ObjectDiscovery(encoder_resolution = (128 , 128) , decoder_resolution = (8 , 8),
                    T = 3 ,  K = 7 , D_slots = 128)
model.to(device)

new_dict = {}
for key in model_dict['model_state_dict']:
    assert key[0 : 7] == 'module.'
    new_dict[key[7 : ]] = model_dict['model_state_dict'][key]
    
model.load_state_dict(new_dict)
# model.load_state_dict(model_dict['model_state_dict'])

model.eval()

for key in model_dict:
    if key == 'model_state_dict' or key == 'optimizer_state_dict':
        continue
    print(f'{key}: {model_dict[key]}')

print("Loaded model!")


Loading model!
Using device: cuda
Initialized ObjectDiscovery!
Initialized ImageEncoder! resolution: (128, 128)
Grid shape: torch.Size([1, 128, 128, 4])
Initialzing SlotAttention parameters: T: 3 , K: 7 , D_slots: 128
Initialized SlotAttentionDecoder! resolution: (8, 8)
Decoder parameters: K: 7 , D_slots: 128
Grid shape: torch.Size([1, 8, 8, 4])
epoch: 179
train_loss: 0.019978486949544468
Loaded model!


In [6]:
from sklearn.metrics.cluster import adjusted_rand_score as sk_ari


# for CLEVERTex11 data


mean_ari = 0 
max_ari = 0
ari_idx = []

with torch.no_grad():
    for idx in tqdm(range(len(val_data))):
        images , true_clusters = val_data[idx]
        images , true_clusters = images.to(device) , true_clusters.to(device)
#         print(images.shape , true_clusters.shape)
        reconstructed , recons , pred_masks  = model(images.unsqueeze(0))
        pred_clusters = pred_masks.squeeze(2).argmax(dim = 1).squeeze(0)
        pred_clusters , true_clusters = np.array(pred_clusters.cpu()).flatten() , np.array(true_clusters.cpu()).flatten()
#         print("prec:" , pred_clusters.shape)
#         print("true:" , true_clusters.shape)
        ari = sk_ari(labels_true = true_clusters , labels_pred = pred_clusters)
        mean_ari += ari
        max_ari = max(max_ari , ari)
        ari_idx.append((ari , idx))


        

print("ARI Score:" , mean_ari / len(val_data))
print("Max ARI Score:" , max_ari)
ari_idx.sort(reverse = True)
top_indices = [index for  _ , index in ari_idx[ : 50]]
print("Top 10 indices with highest ARI scores:", top_indices)


100%|██████████| 10000/10000 [03:19<00:00, 50.10it/s]

ARI Score: 0.5544671070601245
Max ARI Score: 0.9389827038609846
Top 10 indices with highest ARI scores: [7661, 2025, 4159, 7213, 9698, 4058, 8231, 8827, 3018, 573, 9489, 8014, 648, 445, 2267, 4153, 5919, 6539, 3270, 6677, 6906, 4925, 9824, 9774, 8713, 2079, 9915, 5896, 8517, 2865, 6129, 7300, 9470, 862, 8046, 3990, 1429, 6010, 8330, 2160, 9400, 4773, 5506, 9588, 9015, 4917, 5071, 9426, 9254, 4793]


In [7]:
# for CLEVERTex6 data
val_data = ImageDatasetSlotAttention(images_folder = images_val , indices = clever6_indices,
                                    transform = transform, cluster_path = val_cluster_path)
print("image tensor max-min" , val_data[0][0].max().item() , val_data[0][0].min().item())
print("cluster tensor max-min" , val_data[4][1].max().item() , val_data[4][1].min().item())


mean_ari = 0 
max_ari = 0
ari_idx = []

with torch.no_grad():
    for idx in tqdm(range(len(val_data))):
        images , true_clusters = val_data[idx]
        images , true_clusters = images.to(device) , true_clusters.to(device)
#         print(images.shape , true_clusters.shape)
        reconstructed , recons , pred_masks  = model(images.unsqueeze(0))
        pred_clusters = pred_masks.squeeze(2).argmax(dim = 1).squeeze(0)
        pred_clusters , true_clusters = np.array(pred_clusters.cpu()).flatten() , np.array(true_clusters.cpu()).flatten()
#         print("prec:" , pred_clusters.shape)
#         print("true:" , true_clusters.shape)
        ari = sk_ari(labels_true = true_clusters , labels_pred = pred_clusters)
        mean_ari += ari
        max_ari = max(max_ari , ari)
        ari_idx.append((ari , idx))


        

print("ARI Score:" , mean_ari / len(val_data))
print("Max ARI Score:" , max_ari)
ari_idx.sort(reverse = True)
top_indices = [index for  _ , index in ari_idx[ : 50]]
print("Top 10 indices with highest ARI scores:", top_indices)


indices shape: (5319,)
clusters data shape: (10000, 128, 128)
Loading data!


100%|██████████| 5319/5319 [00:11<00:00, 466.11it/s]


Loaded data!
data size: 5319
image tensor max-min 0.8588235378265381 -0.9843137264251709
cluster tensor max-min 6 0


100%|██████████| 5319/5319 [01:47<00:00, 49.29it/s]

ARI Score: 0.5626948637022806
Max ARI Score: 0.9389827038609846
Top 10 indices with highest ARI scores: [4092, 1086, 2219, 3852, 5151, 2167, 4380, 4682, 1619, 313, 5051, 4273, 354, 246, 1212, 2214, 3154, 3504, 1751, 3576, 3687, 2630, 5225, 5196, 4622, 1116, 5278, 3143, 4516, 1526, 3277, 3894, 5038, 467, 4291, 2139, 768, 3210, 4428, 1159, 4998, 2554, 2933, 5099, 4779, 2626, 2710, 5010, 4910, 2566]
